## 1\. Exploração de Dados
>Vamos explorar dados de crédito presentes neste neste link. Os dados estão no formato CSV e contém informações sobre clientes de uma instituição financeira. Em especial, estamos interessados em explicar a segunda coluna, chamada de default, que indica se um cliente é adimplente(default = 0), ou inadimplente (default = 1), ou seja, queremos entender o porque um cliente deixa de honrar com suas dívidas baseado no comportamento de outros atributos, como salário, escolaridade e movimentação financeira. Uma descrição completa dos atributos está abaixo.

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os



df = pd.read_csv('../input/ebac-python-projeto/Python_M10_support material.csv', na_values='na')
df.head(n=10)

### **1.1. Estrutura** 

In [ ]:
df.shape
qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default'] == 0].shape
qtd_inadimplentes, _ = df[df['default'] == 1].shape
print(f"Total de clientes {qtd_total}")
print(f"Total de clientes adimplentes {qtd_adimplentes} ou {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"Total de clientes inadimplentes {qtd_inadimplentes} ou {round(100 * qtd_inadimplentes / qtd_total, 2)}%")

In [ ]:
df.dtypes

### **1.2. Schema** 

 - Atributos **categóricos**.

In [ ]:
df.select_dtypes('object').describe().transpose()

 - Já podemos observar, pelo dataframe transposto acima, que os dados de escolaridade, estado_civil e salario_anual possuem instâncias faltantes e limite_credito e valor_transacoes_12m não são categóricos ambos terão de ser tratados.

 - Atributos **numéricos**.

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

### **1.3. Dados faltantes** 

Podemos verificar quais colunas possuem dados faltantes.

In [ ]:
df.isna().any()

levantando algumas estatisticas sobre as colunas dos dados faltantes.

In [ ]:
def stats_dados_faltantes(df: pd.DataFrame) -> None:

  stats_dados_faltantes = []
  for col in df.columns:
    if df[col].isna().any():
      qtd, _ = df[df[col].isna()].shape
      total, _ = df.shape
      dict_dados_faltantes = {col: {'quantidade': qtd, "porcentagem": round(100 * qtd/total, 2)}}
      stats_dados_faltantes.append(dict_dados_faltantes)

  for stat in stats_dados_faltantes:
    print(stat)

print("Total de dados faltantes: \n")
stats_dados_faltantes(df=df)
print("\n Dados faltantes Adimplentes: \n")
stats_dados_faltantes(df=df[df['default'] == 0])
print("\n Dados faltantes Inadimplentes: \n")
stats_dados_faltantes(df=df[df['default'] == 1])

## 2\. Transformação e limpeza de dados

Agora que conhecemos melhor a natureza do nosso conjunto de dados, vamos conduzir uma atividade conhecida como *data wrangling* que consiste na transformação e limpeza dos dados do conjunto para que possam ser melhor analisados. Em especial, vamos:

 - Corrigir o *schema* das nossas colunas;
 - Remover os dados faltantes.

### **2.1. Correção de schema** 

Na etapa de exploração, notamos que as colunas **limite_credito** e **valor_transacoes_12m** estavam sendo interpretadas como colunas categóricas (`dtype = object`).

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].dtypes

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].head(n=5)

Vamos criar uma função `lambda` para limpar os dados. Modificando o padrão de descrição de números do português para inglês, para que assim o python consiga interpretar os dados como float. Mas antes, vamos testar sua aplicação através do método funcional `map`:

In [ ]:
fn = lambda valor: float(valor.replace(".", "").replace(",", "."))

valores_originais = ['12.691,51', '8.256,96', '3.418,56', '3.313,03', '4.716,22']
valores_limpos = list(map(fn, valores_originais))

print(valores_originais)
print(valores_limpos)

Com a função `lambda` de limpeza pronta, basta aplica-la nas colunas de interesse.

In [ ]:
df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(fn)
df['limite_credito'] = df['limite_credito'].apply(fn)


In [ ]:
df.dtypes

 - Atributos **categóricos**.

In [ ]:
df.select_dtypes('object').describe().transpose()

 - Atributos **numéricos**.

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

Podemos observar que agora os valores de **limite_credito** e **valor_transacoes_12m** encontram-se corretamente identificados.

### **2.2. Remoção de dados faltantes** 

Como o pandas está ciente dos nossos dados faltantes, a remoção das linhas problemáticas é trivial.

In [ ]:
df.dropna(inplace=True)

Vamos analisar a estrutura dos dados novamente.

In [ ]:
qtd_total_novo, _ = df.shape
qtd_adimplentes_novo, _ = df[df['default'] == 0].shape
qtd_inadimplentes_novo, _ = df[df['default'] == 1].shape
print(f"Total de clientes {qtd_total}")
print(f"Novo total de clientes {qtd_total_novo}")
print(f"Total de clientes adimplentes {qtd_adimplentes} ou {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"Novo total de clientes adimplentes {qtd_adimplentes_novo} ou {round(100 * qtd_adimplentes_novo / qtd_total_novo, 2)}%")
print(f"Total de clientes inadimplentes {qtd_inadimplentes} ou {round(100 * qtd_inadimplentes / qtd_total, 2)}%")
print(f"Novo total de clientes inadimplentes {qtd_inadimplentes_novo} ou {round(100 * qtd_inadimplentes_novo / qtd_total_novo, 2)}%")

## 3\. Visualização de dados

Os dados estão prontos, vamos criar diversas visualizações para correlacionar variáveis explicativas com a variável resposta para buscar entender qual fator leva um cliente a inadimplencia. E para isso, vamos sempre comparar a base com todos os clientes com a base de adimplentes e inadimplentes.

Começamos então importando os pacotes de visualização e separando os clientes adimplentes e inadimplentes 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

In [ ]:
df_adimplente = df[df['default'] == 0]
df_inadimplente = df[df['default'] == 1]

### **3.1. Visualizações categóricas** 

Nesta seção, vamos visualizar a relação entre a variável resposta **default** com os atributos categóricos.

In [ ]:
df.select_dtypes('object').head(n=5)

In [ ]:
def grafico_barra_categorico(coluna=str):
    titulos = [f'{coluna} dos Clientes', f'{coluna} dos Clientes Adimplentes', f'{coluna} dos Clientes Inadimplentes']
    eixo = 0
    figura, eixos = plt.subplots(1, 3, figsize=(20, 6), sharex=True)
    plt.tight_layout()
    for dataframe in [df, df_adimplente, df_inadimplente]:
        df_to_plot = dataframe[coluna].value_counts()
        dfd = list(df_to_plot.keys())
        f = sns.barplot(x=dfd, y=df_to_plot[dfd], ax=eixos[eixo])
        f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
        f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)
        

        eixo += 1
    figura.show()


grafico_barra_categorico('escolaridade')
grafico_barra_categorico('salario_anual')

Como é possível observar, a relação parece ser pequena com os dois atributos categoricos vistos acima, talvez uma avaliação por porcentagem seja mais fácil de visualizar.

In [ ]:
from functools import reduce

def grafico_barra_porcentagem(coluna=str):

    dfd = df[coluna].value_counts()
    dfd = list(dfd.keys())

    df_to_adimplente = list(df_adimplente[coluna].value_counts())
    df_to_inadimplente = list(df_inadimplente[coluna].value_counts())
    total_adimplente = reduce(lambda a, b: a + b, df_to_adimplente)
    total_inadimplente = reduce(lambda a, b: a + b, df_to_inadimplente)
    dfpercent_adimplente = list(map(lambda x: (float(x)*100)/total_adimplente, df_to_adimplente))
    dfpercent_inadimplente = list(map(lambda x: (float(x) * 100) / total_inadimplente, df_to_inadimplente))
    total_diff = list(map(lambda a, b: b - a, dfpercent_adimplente,dfpercent_inadimplente))

    f = sns.barplot(x=dfd, y=total_diff)

    f.set(title=f"Diferença na representatividade de inadimplencia por {coluna} ", xlabel=coluna.capitalize(), ylabel='Porcentagem')
    f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)
    plt.tight_layout()


    plt.show()
grafico_barra_porcentagem('escolaridade')

Agora visualizando o comparativo entre a diferença de representatividade de cada escolaridade podemos ver por exemplo, que formados no ensino médio representam 23.72% dos adimplentes mas apenas 21.29% dos inadimplentes, por tanto tendo em vista a distribuição do total, isso pode indicar que possuem menos chances de serem inadimplentes. Mais investigação e maior categorização é necessária, como diferenciar cursos em que as pessoas se graduaram.

### **3.2. Visualizações numéricas** 

Nesta seção, vamos visualizar a relação entre a variável resposta **default** com os atributos numéricos.

In [ ]:
df.drop(['id', 'default'], axis=1).select_dtypes('number').head(n=5)

 - Quantidade de Transações nos Últimos 12 Meses
 - Valor das Transações nos Últimos 12 Meses

In [ ]:
def grafico_hist(coluna = str):
    titulos = [f'{coluna}', f'{coluna} de Adimplentes', f'{coluna} de Inadimplentes']

    eixo = 0
    max_y = 0
    figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

    for dataframe in [df, df_adimplente, df_inadimplente]:

      f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
      f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

      _, max_y_f = f.get_ylim()
      max_y = max_y_f if max_y_f > max_y else max_y
      f.set(ylim=(0, max_y))

      eixo += 1

    figura.show()

grafico_hist('qtd_transacoes_12m')
grafico_hist('valor_transacoes_12m')


Por outro lado conseguimos visualizar uma boa relação, inversamente proporcional, entre a quantia e valor das transações, com a inadimplência. O quê é de se esperar, quem está bem financeiramente já têm uma tendência a não precisar de empréstimos de risco.
Vejamos agora um gráfico relacional entre os dois atributos acima.

In [ ]:
f = sns.relplot(x='valor_transacoes_12m', y='qtd_transacoes_12m', data=df, hue='default')
_ = f.set(
    title='Relação entre Valor e Quantidade de Transações no Último Ano', 
    xlabel='Valor das Transações no Último Ano', 
    ylabel='Quantidade das Transações no Último Ano'
  )

Com o cruzamento dos dois atributos, fica claro que existem um agrupamento de grande volume e valor de transações que está livre de inadimplência. Mostrando que acima de 100 transações e transações acima de 12000 existe um hardcut de adimplência, onde praticamente não é preciso se preocupar com os clientes que se enquadram nesse grupo.